# IMPORT LIBRARY

In [1]:
import re
import pandas as pd
from tqdm import tqdm

# 추출할 col List

In [2]:
col = ['BRCA1',
 'HMGB3',
 'PLXNB2',
 'NOTCH2',
 'RYR2',
 'BTG1',
 'SMC1A',
 'PIM1',
 'FBXW7',
 'CDKN1A',
 'ENPP2',
 'COL6A3',
 'PLEC',
 'SOX9',
 'CLTC',
 'KIF3B',
 'PKHD1',
 'CREBBP',
 'VHL',
 'RNF213',
 'AMOT',
 'TGFBR2',
 'RYR1',
 'SMC3',
 'XDH',
 'AKT1',
 'MYH1',
 'SPOP',
 'CELSR1',
 'SPTAN1',
 'LRP1',
 'YWHAQ',
 'MYH2',
 'ATRX',
 'COL5A1',
 'PIK3CA',
 'SPTA1',
 'NF1',
 'FBN2',
 'RELN',
 'MXRA5',
 'NOTCH1',
 'KMT2D',
 'NLRC5',
 'TSC2',
 'PEG3',
 'VWF',
 'SYMPK',
 'SOWAHC',
 'LAMA1',
 'CASP8',
 'ARID5B',
 'IGSF1',
 'CMPK2',
 'EGFR',
 'NLRP3',
 'IDH2',
 'MAPT',
 'PKD1',
 'TNFAIP3',
 'SCN10A',
 'COL4A1',
 'ZBTB10',
 'TP53',
 'HUWE1',
 'KIT',
 'AXIN1',
 'PTEN',
 'ITIH1',
 'IRF4',
 'RB1',
 'MTOR',
 'FASN',
 'CEBPA',
 'STAT3',
 'BMP2K',
 'LRIG1',
 'EIF4G3',
 'HRAS',
 'GOLGA4',
 'CTCF',
 'COL11A1',
 'TOP2A',
 'CHST2',
 'CHEK2',
 'NCOA6',
 'TM7SF2',
 'COL5A3',
 'NFKB2',
 'MAP3K1',
 'TG',
 'ERCC2',
 'GATA3',
 'CDH8',
 'APC',
 'CTNNB1',
 'ELF3',
 'LAMA2',
 'PTPRD',
 'RET',
 'CDC27',
 'ARHGAP5',
 'PTGIR',
 'MAGI2',
 'MEFV',
 'PTPN11',
 'MYH4',
 'CENPE',
 'NPM1',
 'NFE2L2',
 'DNMT1',
 'BRAF',
 'SYNE1',
 'NCOR2',
 'IDH1',
 'CENPF',
 'COL12A1',
 'EIF1AX',
 'SLC12A4',
 'PIK3R5',
 'DCC',
 'PEX6',
 'B2M',
 'MYLK',
 'BTG2',
 'CACNA1B',
 'PABPC1',
 'FGFR3',
 'CLIP2',
 'CDH1']

# DATA LOAD

In [3]:
train = pd.read_csv("../train_data/train.csv")

In [4]:
sequence = pd.read_csv("../train_data/protein_sequences_output.csv")

In [5]:
train = train[['ID', 'SUBCLASS'] + col]

In [6]:
sequence = sequence[col]

# 함수 정의

In [7]:
# Step 1: 변이를 파싱하고 내림차순으로 정렬하는 함수
def parse_mutations(mutation_str):
    mutation_list = mutation_str.split()  # 변이 문자열을 공백으로 분리하여 리스트로 만듦
    parsed_mutations = []
    for mutation in mutation_list:
        # 정규식을 사용하여 위치를 찾음 (첫 번째 숫자 위치를 추출)
        match = re.search(r'(\d+)', mutation)
        if match:
            position = int(match.group(1))  # 위치를 정수로 변환
            parsed_mutations.append((position, mutation))  # 위치와 변이를 튜플로 저장
    # 위치를 기준으로 내림차순 정렬하여, 뒤쪽 변이를 먼저 처리하도록 만듦
    parsed_mutations.sort(reverse=True, key=lambda x: x[0])
    return [mutation for _, mutation in parsed_mutations]  # 정렬된 변이 목록만 반환

# Step 2: 정렬된 변이들을 기반으로 서열에 적용하는 함수
def apply_complex_mutations(sequence, mutations):
    # 변이가 'WT'라면 그대로 'WT'와 빈 문자열 반환
    if mutations == 'WT':
        return "WT", ""  # 변이된 서열을 'WT'로, 변이 목록은 빈 문자열로 반환

    # 변이를 파싱하여 내림차순으로 정렬된 변이 목록을 가져옴
    mutation_steps = parse_mutations(mutations)
    # 서열을 리스트로 변환하여 인덱스별 수정이 가능하게 준비
    seq_list = list(sequence)
    # 변이를 저장할 mods 딕셔너리: {위치: 변이 내용} 형식으로 저장하여 나중에 한 번에 적용
    mods = {}  
    # 적용된 변이들을 추적하기 위한 리스트, 각 변이를 문자열로 저장
    applied_mutations = []  
    
    for mutation in mutation_steps:
        # 변이를 적용할 때마다 변이 목록에 추가
        applied_mutations.append(mutation)  

        # 삭제 후 삽입 변이 처리
        if "delins" in mutation:
            match = re.match(r"([A-Z])(\d+)_([A-Z])(\d+)delins([A-Z]+)", mutation)
            if match:
                start_pos, end_pos = int(match.group(2)), int(match.group(4))  # 삭제할 위치 범위
                insert_seq = match.group(5)  # 삽입할 아미노산 서열
                for i in range(start_pos, end_pos + 1):
                    mods[i] = ''  # 범위 내 위치를 빈 문자열로 설정하여 삭제
                mods[start_pos] = mods.get(start_pos, "") + insert_seq  # 삽입할 아미노산 저장

        # 삽입 변이 처리
        elif "ins" in mutation:
            match = re.match(r"([A-Z])(\d+)_([A-Z])(\d+)ins([A-Z]+)", mutation)
            if match:
                pos = int(match.group(2))  # 삽입 위치
                insert_seq = match.group(5)  # 삽입할 아미노산 서열
                mods[pos] = mods.get(pos, "") + insert_seq  # 지정된 위치에 아미노산을 삽입

        # 단순 삭제 변이 처리
        elif "del" in mutation:
            match = re.match(r"([A-Z])(\d+)(_([A-Z])(\d+))?del", mutation)
            if match:
                start_pos = int(match.group(2))
                if match.group(4):  # 범위 삭제인 경우
                    end_pos = int(match.group(5))
                    for i in range(start_pos, end_pos + 1):
                        mods[i] = ''  # 범위 내 위치를 빈 문자열로 설정하여 삭제
                else:  # 단일 위치 삭제인 경우
                    mods[start_pos] = ''  # 단일 위치를 빈 문자열로 설정하여 삭제
                    
        # 치환 및 결정종 변이 처리
        elif ">" in mutation:
            match = re.match(r"(\d+)_\d+[A-Z]+>([A-Z]+)(\*)?", mutation)
            if match:
                start_pos = int(match.group(1))
                new_seq = match.group(2)  # 새로운 아미노산 서열
                if match.group(3):  # 치환 후 뒤를 모두 삭제하는 경우
                    mods[start_pos] = new_seq  # 위치에 새로운 아미노산 서열을 치환
                    for i in range(start_pos + 1, len(seq_list) + 1):
                        mods[i] = ''  # 치환된 위치 뒤를 모두 빈 문자열로 설정하여 삭제
                else:
                    mods[start_pos] = new_seq  # 단순 치환 처리

        # 특정 위치 이후를 모두 제거하는 변이 처리
        elif "*" in mutation:
            match = re.match(r"[A-Z](\d+)\*", mutation)
            if match:
                pos = int(match.group(1))
                for i in range(pos, len(seq_list) + 1):
                    mods[i] = ''  # 특정 위치 이후 모든 위치를 빈 문자열로 설정하여 삭제

        # 프레임 변이 처리
        elif "fs" in mutation:
            match = re.match(r"([A-Z])(\d+)fs", mutation)
            if match:
                pos = int(match.group(2))
                mods[pos] = 'Z'  # 위치에 'fs'를 설정
                # 'fs' 이후의 모든 위치를 빈 문자열로 설정하여 삭제
                for i in range(pos + 1, len(seq_list) + 1):
                    mods[i] = ''

        # 단일 치환 변이 처리
        elif re.match(r"^[A-Z]\d+[A-Z]$", mutation):
            pos = int(mutation[1:-1])
            if sequence[pos - 1] == mutation[0]:  # 치환 전 아미노산과 서열이 일치할 경우
                mods[pos] = mutation[-1]  # 지정 위치에 치환할 아미노산 저장

    # Step 3: 저장된 변이를 서열에 한 번에 적용
    result = []
    for i in range(1, len(seq_list) + 1):
        if i in mods:
            result.append(mods[i])  # 변이가 저장된 위치는 해당 변이를 적용
        else:
            result.append(seq_list[i - 1])  # 변이가 없는 위치는 원래 아미노산 유지

    final_sequence = ''.join(result)  # 리스트를 문자열로 변환하여 최종 서열 반환
    mutation_string = " ".join(applied_mutations)  # 적용된 변이들을 " "로 연결하여 변이 목록 반환

    return final_sequence, mutation_string


# 돌연변이 아미노산서열로 교체

In [15]:
# Step 4: 모든 유전자에 대해 변이를 적용하고 최종 서열과 변이 목록을 저장
mutated_sequences_df = pd.DataFrame(index=train.index)
mutation_details_df = pd.DataFrame(index=train.index)

for gene in tqdm(sequence.columns):  # 첫 번째 열은 인덱스이므로 제외
    base_sequence = sequence[gene].iloc[0]  # 각 유전자에 대해 기본 서열을 가져옴
    # apply_complex_mutations 함수를 통해 변이를 적용한 최종 서열과 변이 목록을 각각의 데이터프레임에 저장
    mutated_sequences_df[gene], mutation_details_df[gene] = zip(*train[gene].apply(
        lambda mutation_str: apply_complex_mutations(base_sequence, mutation_str) if mutation_str != 'WT' else ("WT", "")
    ))

# Output: 최종 변이된 서열과 변이 목록을 데이터프레임 형태로 출력
mutated_sequences_df.head(), mutation_details_df.head()


  0%|          | 0/130 [00:00<?, ?it/s]

 77%|███████▋  | 100/130 [00:09<00:02, 13.22it/s]C:\Users\LG\AppData\Local\Temp\ipykernel_14304\3481352599.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mutated_sequences_df[gene], mutation_details_df[gene] = zip(*train[gene].apply(
C:\Users\LG\AppData\Local\Temp\ipykernel_14304\3481352599.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mutated_sequences_df[gene], mutation_details_df[gene] = zip(*train[gene].apply(
C:\Users\LG\AppData\Local\Temp\ipykernel_14304\3481352599.py:8: PerformanceWarning: DataFrame is highly f

(  BRCA1 HMGB3 PLXNB2                                             NOTCH2 RYR2  \
 0    WT    WT     WT                                                 WT   WT   
 1    WT    WT     WT                                                 WT   WT   
 2    WT    WT     WT  MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...   WT   
 3    WT    WT     WT                                                 WT   WT   
 4    WT    WT     WT                                                 WT   WT   
 
   BTG1 SMC1A PIM1 FBXW7 CDKN1A  ... DCC  \
 0   WT    WT   WT    WT     WT  ...  WT   
 1   WT    WT   WT    WT     WT  ...  WT   
 2   WT    WT   WT    WT     WT  ...  WT   
 3   WT    WT   WT    WT     WT  ...  WT   
 4   WT    WT   WT    WT     WT  ...  WT   
 
                                                 PEX6 B2M  \
 0                                                 WT  WT   
 1                                                 WT  WT   
 2  MALAVLRVLEPFPTETPPLAVLLPPGGPWPAAELGLVLALRPAGES...  WT   
 3

In [16]:
mutated_sequences_df.head()

,BRCA1,HMGB3,PLXNB2,NOTCH2,RYR2,BTG1,SMC1A,PIM1,FBXW7,CDKN1A,...,DCC,PEX6,B2M,MYLK,BTG2,CACNA1B,PABPC1,FGFR3,CLIP2,CDH1
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,WT,WT,WT,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,WT,WT,WT,WT,WT,WT,...,WT,MALAVLRVLEPFPTETPPLAVLLPPGGPWPAAELGLVLALRPAGES...,WT,MGDVKLVASSHISKTSLSVDPSRVDSMPLTEAPAFILPPRNLCIKE...,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


# WT 제거 후 돌연 변이 서열 하나의 string으로

In [17]:
# 각 행에서 'WT'를 제거하고 남은 값들을 string로 만듭니다. 단백질은 ,로 구분
df_filtered_strings = mutated_sequences_df.apply(lambda row: ','.join(row[row != 'WT']), axis=1)

# 결과를 새로운 데이터프레임으로 저장합니다.
df_sequence = pd.DataFrame(df_filtered_strings, columns=['Filtered_List'])

# 결과를 확인합니다.
print(df_sequence.head())


                                       Filtered_List
0  MFKKLKQKISEEQQQLQQALAPAQASSNSSTPTRMRSRTSSFTEQL...
1  MTAEPMSESKLNTLVQKLHDFLAHSSEESEETSSPPRLAMNQNTDK...
2  MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...
3  MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...
4  MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...


In [20]:
df_sequence['SUBCLASS'] = train['SUBCLASS']
df_sequence.head()

,Filtered_List,SUBCLASS
0,MFKKLKQKISEEQQQLQQALAPAQASSNSSTPTRMRSRTSSFTEQL...,KIPAN
1,MTAEPMSESKLNTLVQKLHDFLAHSSEESEETSSPPRLAMNQNTDK...,SARC
2,MPALRPALLWALLALWLCCAAPAHALQCRDGYEPCVNEGMCVTYHN...,SKCM
3,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,KIRC
4,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,GBMLGG


In [21]:
# 최종 결과를 CSV 파일로 저장
df_sequence.to_csv('../train_data/train_sequence_tok.csv', index=False)
df_sequence.to_pickle('../train_data/train_sequence_tok.pkl')
